In [ ]:
# setup cell
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent  # since notebooks/ is one level down
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    
from uav_rl_navigation import config
from uav_rl_navigation.utils.env_helpers import make_voyager_for_recording
from stable_baselines3 import PPO

In [ ]:
# import os
# os.system("Xvfb :1 -screen 0 1024x768x24 &")
# os.environ['DISPLAY'] = ':1'

In [4]:
# run episodes and record video
def record_video_voyager(
    model,
    video_env,
    video_length: int,
    deterministic: bool = True,
):

    # reset env before setting camera
    obs = video_env.reset()
    
    # step over the episodes and record video
    for _ in range(video_length):
        action, _ = model.predict(obs, deterministic=deterministic)
        obs, _, _, _ = video_env.step(action)
    video_env.close()


In [5]:
# display video in notebook
from pathlib import Path
import base64
from IPython.display import HTML, display

def show_videos(video_path, prefix):
    """
    Loop over all `video_path/{prefix}*.mp4`, embed each as an
    HTML5 <video> tag with a <figcaption> that shows the filename.
    """
    html_fragments = []
    for mp4 in Path(video_path).glob(f"{prefix}*.mp4"):
        # encode video
        video_b64 = base64.b64encode(mp4.read_bytes()).decode("ascii")
        # get just the filename (or use .stem for name without extension)
        caption = mp4.name
        # build a <figure> with video + caption
        fragment = f"""
        <figure style="margin-bottom: 1em;">
          <figcaption style="font-family: sans-serif; font-size: 0.9em; margin-bottom: 0.3em;">
            {caption}
          </figcaption>
          <video autoplay loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{video_b64}" type="video/mp4" />
            Sorry, your browser doesn’t support embedded videos.
          </video>
        </figure>
        """
        html_fragments.append(fragment)

    display(HTML("<br>".join(html_fragments)))

In [ ]:
# run env and record video
model_path = config.MODELS_DIR / "ppo_waypoint_ts-1_000k.zip"
video_name = "ppo_waypoint_1000kts_2wp"

model = PPO.load(model_path)

video_env = make_voyager_for_recording(
    video_folder=config.VIDEOS_DIR, 
    video_length=500,
    video_name=video_name,
    num_waypoints=1,
    )


# run episodes and record video
record_video_voyager(
    model=model,
    video_env=video_env,
    video_length=500,
    deterministic=True,
)

                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=
                             argv[0]=

Saving video to /Users/user/Documents/education/CAI/bachelor_thesis/uav-rl-navigation/reports/videos/ppo_waypoint_1000kts_2wp-step-0-to-step-500.mp4
MoviePy - Building video /Users/user/Documents/education/CAI/bachelor_thesis/uav-rl-navigation/reports/videos/ppo_waypoint_1000kts_2wp-step-0-to-step-500.mp4.
MoviePy - Writing video /Users/user/Documents/education/CAI/bachelor_thesis/uav-rl-navigation/reports/videos/ppo_waypoint_1000kts_2wp-step-0-to-step-500.mp4



MoviePy - Done !
MoviePy - video ready /Users/user/Documents/education/CAI/bachelor_thesis/uav-rl-navigation/reports/videos/ppo_waypoint_1000kts_2wp-step-0-to-step-500.mp4


In [ ]:
show_videos(config.VIDEOS_DIR, prefix="ppo")